![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/fscampus_small.png?raw=1)

# Derivatives

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
Frankfurt School of Finance & Management

fabian.woebbeking@dozent.frankfurt-school.de

## Case 3: Fixed Income

Case description ...

### Part 1: Basics

The following 3 zero bonds are given:

| Maturity | Price | Redemption |
|:--------:|:-----:|:----------:|
|    1Y    | 95.24 |     100    |
|    2Y    | 89.00 |     100    |
|    3Y    | 81.63 |     100    |

(Table generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate spot and forward rates.

#### Task: Price a three year 10% bond. Calculate the IRR and explain the relationship of the spot rates and the IRR.

#### Task:	Evaluate with the same curve, a three year floater against the one year rate (bond base).

#### Task: Solve analytically and calculate the par coupon rate for a three year bond.

### Part 2: Asset-/ liability-swap

#### Task: Calculate the level of the floating debt (liability swap).

Power has issued a five year bond with a coupon of 2% and a pay out of 100,5. The five year IRS is quoted 2,00 – 2,05 against six month EURIBOR. 

#### Task: Calculate the asset swap.

The Investor has bought a five year AA Bond with a coupon of 3% for 101,00. The 5 Y IRS quotes 2,00 -2,05 against 6M Euribor. 

### Part 3: Single curve

The following swap rates are given: 

| Maturity | 12M Euribor | Eonia |
|:--------:|:-----------:|:-----:|
|    1Y    |    4.00%    | 3.30% |
|    2Y    |    5.00%    | 4.50% |
|    3Y    |    6.00%    | 5.30% |

(Table generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate all spot and forward rates using the single curve approach.

#### Task: Price a three year swap against the one year rate using the single curve approach.

#### Task: Price the forward swap in one for two. Show that the result is correct and compare it to the zero rate in one for two using the single curve approach. 

#### Task: Mark to market a swap, 8% against 12M Euribor with a remaining maturity of 3 years and EUR 100mln notional.

### Part 4: Multi curve

Consider a three year swap against the one year rate using the multi curve approach. The following swap rates are given: 

| Maturity | 12M Euribor | Eonia |
|:--------:|:-----------:|:-----:|
|    1Y    |    4.00%    | 3.30% |
|    2Y    |    5.00%    | 4.50% |
|    3Y    |    6.00%    | 5.30% |

(Table generated with [TablesGenerator](https://www.tablesgenerator.com/markdown_tables))

#### Task: Calculate the spot rates.

#### Task: Calculate the implied forward rates. 

(Hint: the one year spot rate for the swap cash flow is taken from the single curve approach.)


#### Task: Calculate the forward swap in one for two.


### Part 5: Interest rate parity / FX forward

#### Task: Calculate the one year (365 days) FX swap forward using covered interest rate parity. 
* Spot = 1,3193 
* USD interest = 1,05310%
* EUR interest = 1,4375%

#### Task: Mark to market the following cross currency swap.

The term structure of interest rates is flat at 6% in USD and 8% in EUR (continuous rates). The current exchange rate is 1.5625 EURUSD. You pay 7% in EUR and receive 3% in USD, with principals being USD 15mln and EUR 25mln. Payments are exchanged annually with 3 payments left (3 years to maturity).